**Querying postgreSQL in Jupyter notebook**

Useful for writing notes and iterating over SQL queries. I kept an example down below to show how queries can evolve.

In [1]:
import pandas as pd
import sqlalchemy
import sqlalchemy_utils
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

Enter the database:
<br>
`psql baseball -h localhost -U lacar`

Get all columns of a table
<br>
`\d+ my_table`

Find the tables of the database:
<br>
`\dt`

# Order of execution

1. FROM /JOIN (subqueries)
2. WHERE
3. GROUP BY
4. HAVING
5. SELECT
6. DISTINCT
7. ORDER BY
8. LIMIT/OFFSET

It can be summarized as:
**calling -> aggregating -> displaying -> filter**

<br>
- First creates a working dataset, then filters with rows added by conditions
<br>
- Column name aliases are not accessed for all commands except after SELECT (aliases for table names are okay) (postgreSQL seems to have some exceptions for SELECT with column name aliasing)

# Skills progression

Descriptions of beginner, intermediate, and advanced SQL skills from [this link](https://softwareengineering.stackexchange.com/questions/181651/are-these-sql-concepts-for-beginners-intermediate-or-advanced-developers)
    
    


# Example of formatting when using a Python function

In [66]:
def return_df_metric_rate_batter(metric, batter_id):

    sql_query = """
    SELECT "game_date", "batter", "events" FROM statcast
    WHERE "batter"= 
    """ + str(batter_id) + """
    AND "game_date" BETWEEN '2019-03-28' AND '2019-04-30'
    AND "events" IS NOT NULL
    """

    df_events = pd.read_sql_query(sql_query,con)
    
    df_summary = df_events.groupby('events').count()['game_date'] / df_events.count()[0]
    return df_summary[metric]

# Date-related tips

from Xufeng

[id], [datetime]
1, 2009-8-1
2, 2003-9-1

select *
from test_table
where datetime > '2008-1-1'
;

select *
from test_table
where year(datetime) > 2008
;

if the table has an index on datetime, Q2 will be much slower than Q1
making query SARGable (Search Aurgmentable)

# Using SQL with baseball database

Each case poses a problem and then I iterate my queries until I get the desired answer. Some addressed and possible questions are:

- What teams have done best against the shift?
- Which pitchers have thrown the top 3 most pitches for each day?
- What ballparks have the highest HR rate?
- Which pitchers have the highest whiff rate?
- What pitch for each pitcher induces the most swing and misses?
- What pitch for each pitcher *precedes* the pitch that causes the most swing and misses?

In [2]:
# Define a database name
# Set your postgres username
dbname = "baseball"
username = "lacar"  # change this to your username

In [3]:
# Working with PostgreSQL in Python
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database=dbname, user=username)

In [4]:
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine("postgres://%s@localhost/%s" % (username, dbname))
print(engine.url)

postgres://lacar@localhost/baseball


# Initial  queries

## Show what statcast table looks like

In [5]:
sql_query = """
SELECT * FROM statcast
LIMIT 10
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   level_0  index pitch_type  game_date  release_speed  release_pos_x  \
0   292328   5971         SL 2018-07-15           78.3         2.9663   
1   292329   5986         FF 2018-07-15           93.3         2.4882   
2   292330   5998         CH 2018-07-15           81.0         3.2677   
3   292331   6013         SI 2018-07-15           90.0         3.4771   
4   292332   6032         FF 2018-07-15           90.8         2.5508   
5   292333   6048         SL 2018-07-15           84.4         1.7845   
6   292334   6062         CU 2018-07-15           79.5         1.8425   
7   292335   6075         SL 2018-07-15           84.8         1.7716   
8   292336   6090         FT 2018-07-15           94.5         1.7428   
9   292337   6105         SL 2018-07-15           86.0         1.7914   

   release_pos_z    player_name    batter   pitcher  ... home_score  \
0         4.4622   Adam Kolarek  543257.0  592473.0  ...        1.0   
1         5.4060   Adam Kolarek  543257.0  592473.0  .

## Make a simpler table for querying

In [6]:
sql_query = """
SELECT pitch_type, game_date, release_speed, home_team, away_team, batter, pitcher, player_name, balls, strikes, home_score, away_score 
FROM statcast
LIMIT 25000
"""

df_query = pd.read_sql_query(sql_query, con)
print(df_query)

df_query.to_sql('statcast_simple', engine, if_exists='replace')

      pitch_type  game_date  release_speed home_team away_team    batter  \
0             SL 2018-07-25           82.8        TB       NYY  622110.0   
1             FT 2018-07-25           93.1        TB       NYY  622110.0   
2             FF 2018-07-25           92.4        TB       NYY  622110.0   
3             FF 2018-07-25           92.5        TB       NYY  595281.0   
4             SL 2018-07-25           89.8        TB       NYY  519317.0   
...          ...        ...            ...       ...       ...       ...   
24995         CH 2018-07-15           88.2       MIN        TB  621563.0   
24996         FF 2018-07-15           95.7       MIN        TB  543068.0   
24997         CH 2018-07-15           90.9       MIN        TB  543068.0   
24998         SL 2018-07-15           86.6       MIN        TB  543068.0   
24999         FT 2018-07-15           94.5       MIN        TB  543068.0   

        pitcher      player_name  balls  strikes  home_score  away_score  
0      57066

Selecting based on column value (note single quote marks)
<br>
`SELECT * FROM player_id
WHERE name_last='machado';`

Selecting based on two column values
<br>
`SELECT * FROM player_id
WHERE name_last='machado'
AND name_first='manny';`

Count the number of games that Machado has played 3B in 2019
<br>
Machado's MLB key: 592518

Tried several things

Gives all pitches:
`SELECT COUNT("game_date") FROM statcast
WHERE "fielder_5"=592518 AND "game_date"
BETWEEN '2019-01-01' AND '2019-12-31' LIMIT 5;`

To get unique game dates:
`SELECT DISTINCT "game_date" FROM statcast WHERE "fielder_5"=592518 AND "game_date" BETWEEN '2019-01-01' AND '2019-12-31' LIMIT 5;`

To get number of unique game dates (need the table AS):
<br>
`SELECT COUNT(*) FROM (SELECT DISTINCT "game_date" FROM statcast WHERE "fielder_5"=592518 AND "game_date" BETWEEN '2019-01-01' AND '2019-12-31' LIMIT 5) AS machado_3b_games;`

... and without limit
<br>
`SELECT COUNT(*) FROM (SELECT DISTINCT "game_date" FROM statcast WHERE "fielder_5"=592518 AND "game_date" BETWEEN '2019-01-01' AND '2019-12-31') AS machado_3b_games;`
 
 `count 
   119
(1 row)`

Games at SS:
<br>
`SELECT COUNT(*) FROM (SELECT DISTINCT "game_date" FROM statcast WHERE "fielder_6"=592518 AND "game_date" BETWEEN '2019-01-01' AND '2019-12-31') AS machado_3b_games;`
<br>
`count 
    37
(1 row)`




## Get Padres stats for 2019

In [130]:
sql_query = """
SELECT "game_date", "batter", "events", "home_team" FROM statcast
WHERE "game_date" BETWEEN '2019-03-28' AND '2019-04-30'
AND "events" IS NOT NULL
AND "home_team"='SD'
LIMIT 10
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date    batter                     events home_team
0 2019-04-24  570267.0  grounded_into_double_play        SD
1 2019-04-24  596129.0                     single        SD
2 2019-04-24  571745.0                  field_out        SD
3 2019-04-24  571976.0                  strikeout        SD
4 2019-04-24  665487.0                  force_out        SD
5 2019-04-24  594824.0                     single        SD
6 2019-04-24  642336.0                  field_out        SD
7 2019-04-24  429665.0                  strikeout        SD
8 2019-04-24  605480.0                  field_out        SD
9 2019-04-24  592387.0                  field_out        SD


## Get Padres away batting average for 2019

In [73]:
# Get total number of hits divided by total number of plate appearances 

# Get total number of plate appearances
sql_query = """
SELECT 
    EXTRACT(YEAR FROM game_date) AS season, 
    COUNT(events) AS total_plate_appearances
FROM statcast
WHERE away_team='SD'
AND game_date BETWEEN '2019-04-01' AND '2019-06-01'
AND events IS NOT NULL
GROUP BY season;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   season  total_plate_appearances
0  2019.0                     2097


In [72]:
# Get total number of hits
sql_query = """
SELECT 
    EXTRACT(YEAR FROM game_date) AS season, 
    COUNT(events) AS hits
FROM statcast
WHERE away_team='SD'
AND game_date BETWEEN '2019-04-01' AND '2019-06-01'
AND events IN ('single', 'double')
GROUP BY season;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   season  count
0  2019.0    372


In [81]:
# Use a self join to get hits and plate appearances in the same table

sql_query = """
SELECT hits_table.season, 
       hits_table.hits, 
       total_pa_table.total_pas,
       hits_table.hits::decimal/total_pa_table.total_pas AS imperfect_obp
FROM 
    (SELECT 
        EXTRACT(YEAR FROM game_date) AS season, 
        COUNT(events) AS hits
    FROM statcast
    WHERE away_team='SD'
    AND inning_topbot='Top'
    AND game_date BETWEEN '2018-04-01' AND '2019-10-01'
    AND events IN ('single', 'double', 'triple', 'home_run')
    GROUP BY season) AS hits_table
JOIN
    (SELECT 
        EXTRACT(YEAR FROM game_date) AS season, 
        COUNT(events) AS total_pas
    FROM statcast
    WHERE away_team='SD'
    AND inning_topbot='Top'
    AND game_date BETWEEN '2018-04-01' AND '2019-10-01'
    AND events IS NOT NULL
    GROUP BY season) AS total_pa_table
ON hits_table.season=total_pa_table.season;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   season  hits  total_pas  imperfect_obp
0  2018.0   666       3026       0.220093
1  2019.0   688       3091       0.222582


## Get batters of average WAR (Renfroe like) 2019

In [62]:
sql_query = """
SELECT "Name", "WAR" FROM batting_stats
WHERE "WAR" BETWEEN 1.5 and 2.5
"""

df_query = pd.read_sql_query(sql_query,con)
    
print(df_query)

                  Name  WAR
0            Josh Bell  2.5
1         Kole Calhoun  2.5
2    Edwin Encarnacion  2.5
3          Khris Davis  2.5
4          Nelson Cruz  2.5
..                 ...  ...
231      Freddy Galvis  1.5
232       Amed Rosario  1.5
233    Kevin Kiermaier  1.5
234        Kyle Seager  1.5
235       Juan Lagares  1.5

[236 rows x 2 columns]


In [ ]:
sql_query = """
SELECT "Name", "WAR" FROM batting_stats
WHERE "Name" LIKE '%Renfroe'
LIMIT 10
"""

df_query = pd.read_sql_query(sql_query,con)    
print(df_query)


## Get Pitchers of average WAR (1.9-2) 2019

In [132]:
sql_query = """
SELECT "Name", "WAR", "Season" FROM pitching_stats
WHERE "WAR" BETWEEN 1.9 AND 2
AND "Season"=2019
"""

df_query = pd.read_sql_query(sql_query,con)
#print(df_query)
df_query

,Name,WAR,Season
0,Domingo German,2.0,2019.0
1,Wade Miley,2.0,2019.0
2,Brett Anderson,2.0,2019.0
3,Merrill Kelly,2.0,2019.0
4,Ivan Nova,2.0,2019.0
5,Tanner Roark,2.0,2019.0
6,Aroldis Chapman,2.0,2019.0
7,Martin Perez,1.9,2019.0
8,Trent Thornton,1.9,2019.0
9,Daniel Norris,1.9,2019.0


## Get no. of at-bats that Tatis faced a two strike count

In [135]:
# Get Tatis keyalbam
sql_query = """
SELECT * FROM player_id
WHERE name_last='tatis';
"""
df_query = pd.read_sql_query(sql_query,con)    
print(df_query)

   index name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0   1738     tatis   fernando     665487  tatif002  tatisfe02          19709   

   mlb_played_first  mlb_played_last  
0            2019.0           2019.0  


In [ ]:
# Get all at-bats hard coding the keymlbam

In [138]:
sql_query = """
SELECT "strikes", "batter", "name_last" FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam='665487'
LIMIT 5
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   strikes    batter name_last
0      0.0  665487.0     tatis
1      0.0  665487.0     tatis
2      0.0  665487.0     tatis
3      2.0  665487.0     tatis
4      1.0  665487.0     tatis


### Get all at-bats using a subquery, searching for "Tatis"

In [7]:
sql_query = """
SELECT COUNT("strikes") FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   count
0    393


### Count number of Tatis at-bats

In [142]:
sql_query = """
SELECT COUNT("events") FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "events" IS NOT NULL
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   count
0    371


## Get the number of pitches of each pitch type that Tatis faced on two strike counts

In [150]:
sql_query = """
SELECT pitch_type, COUNT(*) AS no_pitches
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2
GROUP BY pitch_type
ORDER BY no_pitches DESC
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  no_pitches
0         FF         126
1         SL          97
2         CH          57
3         CU          34
4         SI          22
5         FT          21
6         FC          17
7         KC          14
8         FS           5


## Get the breakdown in proportion of pitch types that Tatis faced on two strike counts

**Return to this**

In [195]:
# Just get total no two strike counts
sql_query = """
SELECT COUNT(*) 
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   count
0    393


In [196]:
sql_query = """
SELECT pitch_type, COUNT(*) AS no_pitches
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2
GROUP BY pitch_type
ORDER BY no_pitches DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  no_pitches
0         FF         126
1         SL          97
2         CH          57
3         CU          34
4         SI          22
5         FT          21
6         FC          17
7         KC          14
8         FS           5


**Complex query**

- use of CTE
- note notation, where commas are
- note use of decimal format 

In [210]:
sql_query = """
WITH
    total_table AS
        (SELECT COUNT(*) AS total_pitches
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2),
        
    pitch_counts_table AS
        (SELECT pitch_type, COUNT(*) AS no_pitches
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2
        GROUP BY pitch_type
        ORDER BY no_pitches DESC)
        
SELECT
    pitch_counts_table.pitch_type, 
    pitch_counts_table.no_pitches,
    pitch_counts_table.no_pitches::decimal / total_table.total_pitches AS pitch_type_proportion
FROM pitch_counts_table, total_table;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  no_pitches  pitch_type_proportion
0         FF         126               0.320611
1         SL          97               0.246819
2         CH          57               0.145038
3         CU          34               0.086514
4         SI          22               0.055980
5         FT          21               0.053435
6         FC          17               0.043257
7         KC          14               0.035623
8         FS           5               0.012723


In [236]:
# Write a SQL query to find all duplicate emails in a table named Person.

sql_query = """
SELECT pitch_type, game_date, player_name
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    pitch_type  game_date      player_name
0           FF 2019-04-23     Erik Swanson
1           FF 2019-04-23     Erik Swanson
2           SL 2019-04-23     Erik Swanson
3           SL 2019-04-23     Erik Swanson
4           FF 2019-04-23     Erik Swanson
..         ...        ...              ...
388         FF 2019-04-26     Max Scherzer
389         CH 2019-04-24  Felix Hernandez
390         SI 2019-04-23     Cory Gearrin
391         SL 2019-04-23     Cory Gearrin
392         CH 2019-04-24  Felix Hernandez

[393 rows x 3 columns]


## Get the average speed of a pitch using a window function

**using window functions**

In [237]:
# base query
sql_query = """
SELECT pitch_type, game_date, player_name, release_speed
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    pitch_type  game_date      player_name  release_speed
0           FF 2019-04-23     Erik Swanson           93.5
1           FF 2019-04-23     Erik Swanson           93.5
2           SL 2019-04-23     Erik Swanson           85.6
3           SL 2019-04-23     Erik Swanson           86.0
4           FF 2019-04-23     Erik Swanson           93.9
..         ...        ...              ...            ...
388         SL 2019-04-26     Max Scherzer           84.9
389         FF 2019-04-26     Max Scherzer           95.8
390         CH 2019-04-24  Felix Hernandez           82.6
391         SI 2019-04-23     Cory Gearrin           92.1
392         SL 2019-04-23     Cory Gearrin           85.0

[393 rows x 4 columns]


In [240]:
# Get average using group by, ignore player name
sql_query = """
SELECT pitch_type, AVG(release_speed) AS avg_speed
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2
        GROUP BY pitch_type;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  avg_speed
0         CH  84.650877
1         CU  78.308824
2         FC  88.235294
3         FF  93.530952
4         FS  82.380000
5         FT  91.728571
6         KC  82.078571
7         SI  92.922727
8         SL  85.253608


In [244]:
# Get average using window/over()
sql_query = """
SELECT pitch_type, release_speed, AVG(release_speed) OVER() AS avg_speed 
        FROM statcast
        JOIN player_id
        ON statcast.batter=player_id.key_mlbam
        WHERE key_mlbam=
            (SELECT key_mlbam FROM player_id
            WHERE name_last='tatis'
            AND name_first='fernando')
        AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    pitch_type  release_speed  avg_speed
0           FF           93.5  87.973791
1           FF           93.5  87.973791
2           SL           85.6  87.973791
3           SL           86.0  87.973791
4           FF           93.9  87.973791
..         ...            ...        ...
388         FF           95.8  87.973791
389         SI           92.1  87.973791
390         CH           82.6  87.973791
391         CH           85.5  87.973791
392         SL           85.0  87.973791

[393 rows x 3 columns]


In [246]:
# Note above that the over just did for all pitches - use partition to bin by each pitch

# Get average using window/over()
sql_query = """
SELECT pitch_type, player_name, release_speed, AVG(release_speed) OVER(PARTITION BY player_name, pitch_type) AS avg_speed 
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    pitch_type       player_name  release_speed  avg_speed
0           FF   Adam Wainwright           88.5      88.50
1           SI   Adam Wainwright           90.6      90.60
2           SL     Adrian Houser           86.3      86.30
3           SL  Andrew Kittredge           89.1      88.30
4           SL  Andrew Kittredge           87.5      88.30
..         ...               ...            ...        ...
388         SL      Zack Greinke           82.0      82.86
389         SL      Zack Greinke           83.8      82.86
390         SL      Zack Greinke           82.6      82.86
391         SL      Zack Greinke           81.1      82.86
392         SL      Zack Greinke           84.8      82.86

[393 rows x 4 columns]


In [23]:
# Note above that the over just did for all pitches - use partition to bin by each pitch

# Get average using window/over()
sql_query = """
SELECT pitch_type, player_name, release_speed, MAX(release_speed) OVER(PARTITION BY player_name, pitch_type) AS max_speed 
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    pitch_type       player_name  release_speed  max_speed
0           FF   Adam Wainwright           88.5       88.5
1           SI   Adam Wainwright           90.6       90.6
2           SL     Adrian Houser           86.3       86.3
3           SL  Andrew Kittredge           89.1       89.1
4           SL  Andrew Kittredge           87.5       89.1
..         ...               ...            ...        ...
388         SL      Zack Greinke           82.0       84.8
389         SL      Zack Greinke           83.8       84.8
390         SL      Zack Greinke           81.1       84.8
391         SL      Zack Greinke           82.6       84.8
392         SL      Zack Greinke           84.8       84.8

[393 rows x 4 columns]


In [25]:
# DOESN'T WORK SINCE IT USED AN ALIAS - use partition to bin by each pitch

# Get average using window/over()
sql_query = """
SELECT pitch_type, player_name, release_speed, MAX(release_speed) OVER(PARTITION BY player_name, pitch_type) AS max_speed 
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2
AND release_speed=max_speed;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

DatabaseError: Execution failed on sql '
SELECT pitch_type, player_name, release_speed, MAX(release_speed) OVER(PARTITION BY player_name, pitch_type) AS max_speed 
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "strikes" = 2
AND release_speed=MAX(release_speed) OVER(PARTITION BY player_name, pitch_type);
': window functions are not allowed in WHERE
LINE 11: AND release_speed=MAX(release_speed) OVER(PARTITION BY playe...
                           ^


## Making comparisons on a date-to-date basis (revisit)

In [262]:
sql_query = """
SELECT game_date, TO_DAYS(game_date), inning, strikes FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "events" IS NOT NULL
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

DatabaseError: Execution failed on sql '
SELECT game_date, TO_DAYS(game_date), inning, strikes FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND "events" IS NOT NULL
LIMIT 10;
': function to_days(timestamp without time zone) does not exist
LINE 2: SELECT game_date, TO_DAYS(game_date), inning, strikes FROM s...
                          ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


## Using window function for sum

In [273]:
# First show the count - just first 20 rows
sql_query = """
SELECT * FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    level_0  index pitch_type  game_date  release_speed  release_pos_x  \
0    730400  14598         FC 2019-03-28           86.2         2.6059   
1    730399  14595         FC 2019-03-28           86.4         2.7543   
2    730398  14570         FF 2019-03-28           92.1         2.6641   
3    730397  14559         FC 2019-03-28           86.0         2.7570   
4    730337  13655         FC 2019-03-28           85.1         2.8588   
5    730276  12747         FF 2019-03-28           91.3         2.8440   
6    730275  12738         CU 2019-03-28           78.7         2.8329   
7    730274  12723         CU 2019-03-28           79.4         2.9261   
8    730273  12703         FC 2019-03-28           85.0         2.8221   
9    726923  10892         SL 2019-03-29           81.9         3.1131   
10   726922  10884         SL 2019-03-29           81.9         3.3420   
11   726853  10333         SL 2019-03-29           79.5         3.0098   
12   726852  10324         SI 2019-03-

In [274]:
# First show the count - just first 20 rows

# level_0 is acting as an index
sql_query = """
SELECT level_0, game_date, inning, balls, strikes FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    level_0  game_date  inning  balls  strikes
0    730400 2019-03-28     2.0    0.0      0.0
1    730399 2019-03-28     2.0    0.0      1.0
2    730398 2019-03-28     2.0    1.0      1.0
3    730397 2019-03-28     2.0    2.0      1.0
4    730337 2019-03-28     4.0    0.0      0.0
5    730276 2019-03-28     7.0    0.0      0.0
6    730275 2019-03-28     7.0    0.0      1.0
7    730274 2019-03-28     7.0    0.0      2.0
8    730273 2019-03-28     7.0    1.0      2.0
9    726923 2019-03-29     2.0    0.0      0.0
10   726922 2019-03-29     2.0    1.0      0.0
11   726853 2019-03-29     4.0    0.0      0.0
12   726852 2019-03-29     4.0    1.0      0.0
13   726851 2019-03-29     4.0    2.0      0.0
14   726850 2019-03-29     4.0    2.0      1.0
15   726849 2019-03-29     4.0    3.0      1.0
16   726848 2019-03-29     4.0    3.0      2.0
17   726784 2019-03-29     6.0    0.0      0.0
18   726783 2019-03-29     6.0    1.0      0.0
19   726782 2019-03-29     6.0    2.0      0.0


In [57]:
# Make running sum of number of balls - just first 20 rows - not sure if this exactly I intended
sql_query = """
SELECT s.game_date, s.inning, s.balls, s.strikes, SUM(balls) OVER(PARTITION BY game_date) AS total_balls_for_date
FROM statcast AS s
JOIN player_id
ON s.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND s.inning<2
ORDER BY s.game_date, s.inning, s.balls, s.strikes
LIMIT 40;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    game_date  inning  balls  strikes  total_balls_for_date
0  2019-04-02     1.0    0.0      0.0                   1.0
1  2019-04-02     1.0    0.0      1.0                   1.0
2  2019-04-02     1.0    0.0      2.0                   1.0
3  2019-04-02     1.0    1.0      2.0                   1.0
4  2019-04-13     1.0    0.0      0.0                   0.0
5  2019-04-13     1.0    0.0      1.0                   0.0
6  2019-04-13     1.0    0.0      2.0                   0.0
7  2019-04-18     1.0    0.0      0.0                   8.0
8  2019-04-18     1.0    0.0      1.0                   8.0
9  2019-04-18     1.0    1.0      1.0                   8.0
10 2019-04-18     1.0    2.0      1.0                   8.0
11 2019-04-18     1.0    2.0      2.0                   8.0
12 2019-04-18     1.0    3.0      2.0                   8.0
13 2019-04-19     1.0    0.0      0.0                   2.0
14 2019-04-19     1.0    0.0      1.0                   2.0
15 2019-04-19     1.0    1.0      1.0   

In [65]:
# Make running sum of number of balls - just first 20 rows - not sure if this exactly I intended
sql_query = """
SELECT s.game_date, s.inning AS i_alias, s.balls, s.strikes, SUM(balls) OVER(PARTITION BY game_date) AS total_balls_for_date
FROM statcast AS s
JOIN player_id
ON s.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND s.i_alias < 2
ORDER BY s.game_date, s.balls, s.strikes
LIMIT 40;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

DatabaseError: Execution failed on sql '
SELECT s.game_date, s.inning AS i_alias, s.balls, s.strikes, SUM(balls) OVER(PARTITION BY game_date) AS total_balls_for_date
FROM statcast AS s
JOIN player_id
ON s.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND s.i_alias < 2
ORDER BY s.game_date, s.balls, s.strikes
LIMIT 40;
': column s.i_alias does not exist
LINE 10: AND s.i_alias < 2
             ^


In [37]:
# Make running sum of number of balls - just first 20 rows - not sure if this exactly I intended
sql_query = """
SELECT inning AS i_alias, AVG(balls)
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
GROUP BY i_alias
LIMIT 40;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    i_alias       avg
0       1.0  0.869565
1       2.0  0.879032
2       3.0  1.028708
3       4.0  0.878261
4       5.0  0.887931
5       6.0  0.806897
6       7.0  0.985714
7       8.0  0.938650
8       9.0  0.992593
9      10.0  1.000000
10     11.0  1.500000
11     12.0  0.500000


## Using window function, rank - show pitch speed and rank

In [ ]:
RANK() OVER (
    [PARTITION BY partition_expression, ... ]
    ORDER BY sort_expression [ASC | DESC], ...
)

In [281]:
# First show release speed - just first 10 rows
sql_query = """
SELECT player_name, release_speed FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

         player_name  release_speed
0  Madison Bumgarner           86.2
1  Madison Bumgarner           86.4
2  Madison Bumgarner           92.1
3  Madison Bumgarner           86.0
4  Madison Bumgarner           85.1
5  Madison Bumgarner           91.3
6  Madison Bumgarner           78.7
7  Madison Bumgarner           79.4
8  Madison Bumgarner           85.0
9      Derek Holland           81.9


In [26]:
# Using OVER() with ORDER BY
sql_query = """
SELECT player_name, release_speed, 
    RANK() OVER(ORDER BY release_speed DESC)
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

          player_name  release_speed  rank
0   Madison Bumgarner           86.2   919
1   Madison Bumgarner           86.4   903
2   Madison Bumgarner           92.1   440
3   Madison Bumgarner           86.0   928
4   Madison Bumgarner           85.1   996
5   Madison Bumgarner           91.3   531
6   Madison Bumgarner           78.7  1315
7   Madison Bumgarner           79.4  1295
8   Madison Bumgarner           85.0  1006
9       Derek Holland           81.9  1196
10      Derek Holland           81.9  1196
11      Derek Holland           79.5  1291
12      Derek Holland           91.7   490
13      Derek Holland           90.8   580
14      Derek Holland           81.5  1213
15      Derek Holland           91.1   553
16      Derek Holland           91.3   531
17        Trevor Gott           79.7  1281
18        Trevor Gott           79.6  1285
19        Trevor Gott           93.3   309


In [42]:
# Using OVER() with ORDER BY
sql_query = """
SELECT *
FROM statcast_simple
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    index pitch_type  game_date  release_speed home_team away_team    batter  \
0       0         FT 2017-06-09           89.7        TB       OAK  446334.0   
1       1         FC 2017-06-09           83.8        TB       OAK  446334.0   
2       2         CH 2017-06-09           82.6        TB       OAK  572816.0   
3       3         FT 2017-06-09           90.0        TB       OAK  605480.0   
4       4         FT 2017-06-09           89.9        TB       OAK  605480.0   
5       5         FT 2017-06-09           89.8        TB       OAK  605480.0   
6       6         FT 2017-06-09           89.9        TB       OAK  605480.0   
7       7         FT 2017-06-09           90.2        TB       OAK  491696.0   
8       8         FT 2017-06-09           89.8        TB       OAK  491696.0   
9       9         FC 2017-06-09           84.9        TB       OAK  491696.0   
10     10         FT 2017-06-09           89.3        TB       OAK  621002.0   
11     11         FC 2017-06-09         

In [50]:
# Using OVER() with ORDER BY
sql_query = """
SELECT pitcher, AVG(release_speed)
FROM statcast_simple
GROUP BY pitcher;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      pitcher        avg
0    606930.0  94.385106
1    621381.0  90.830612
2    600526.0  93.830000
3    519076.0  90.933333
4    621121.0  89.980220
..        ...        ...
234  593423.0  95.440741
235  542947.0  87.387500
236  450172.0  87.234545
237  571871.0  89.716667
238  547179.0  91.717647

[239 rows x 2 columns]


In [27]:
# Using OVER() with ORDER BY, partitioning by player_name - with RANK() (not DENSE_RANK)
sql_query = """
SELECT player_name, release_speed, 
    RANK() OVER(PARTITION BY player_name ORDER BY release_speed DESC) pitchers_speed_rank
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY player_name, pitchers_speed_rank
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  release_speed  pitchers_speed_rank
0       A.J. Minter           97.3                    1
1       A.J. Minter           92.6                    2
2       A.J. Minter           92.2                    3
3   Adam Wainwright           91.0                    1
4   Adam Wainwright           90.9                    2
5   Adam Wainwright           90.7                    3
6   Adam Wainwright           90.6                    4
7   Adam Wainwright           88.5                    5
8   Adam Wainwright           88.0                    6
9   Adam Wainwright           85.7                    7
10  Adam Wainwright           85.2                    8
11  Adam Wainwright           82.2                    9
12  Adam Wainwright           75.6                   10
13  Adam Wainwright           75.2                   11
14  Adam Wainwright           74.7                   12
15    Adrian Houser           87.7                    1
16    Adrian Houser           86.3              

In [28]:
# Using OVER() with ORDER BY, partitioning by player_name
sql_query = """
SELECT player_name, release_speed, 
    DENSE_RANK() OVER(PARTITION BY player_name ORDER BY release_speed DESC) pitchers_speed_rank
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY player_name, pitchers_speed_rank
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  release_speed  pitchers_speed_rank
0       A.J. Minter           97.3                    1
1       A.J. Minter           92.6                    2
2       A.J. Minter           92.2                    3
3   Adam Wainwright           91.0                    1
4   Adam Wainwright           90.9                    2
5   Adam Wainwright           90.7                    3
6   Adam Wainwright           90.6                    4
7   Adam Wainwright           88.5                    5
8   Adam Wainwright           88.0                    6
9   Adam Wainwright           85.7                    7
10  Adam Wainwright           85.2                    8
11  Adam Wainwright           82.2                    9
12  Adam Wainwright           75.6                   10
13  Adam Wainwright           75.2                   11
14  Adam Wainwright           74.7                   12
15    Adrian Houser           87.7                    1
16    Adrian Houser           86.3              

In [30]:
# Using OVER() with ORDER BY, partitioning by player_name - mimicking what ties would look like
sql_query = """
SELECT player_name, ROUND(release_speed), 
    RANK() OVER(PARTITION BY player_name ORDER BY ROUND(release_speed) DESC) pitchers_speed_rank
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY player_name, pitchers_speed_rank
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  round  pitchers_speed_rank
0       A.J. Minter   97.0                    1
1       A.J. Minter   93.0                    2
2       A.J. Minter   92.0                    3
3   Adam Wainwright   91.0                    1
4   Adam Wainwright   91.0                    1
5   Adam Wainwright   91.0                    1
6   Adam Wainwright   91.0                    1
7   Adam Wainwright   88.0                    5
8   Adam Wainwright   88.0                    5
9   Adam Wainwright   86.0                    7
10  Adam Wainwright   85.0                    8
11  Adam Wainwright   82.0                    9
12  Adam Wainwright   76.0                   10
13  Adam Wainwright   75.0                   11
14  Adam Wainwright   75.0                   11
15    Adrian Houser   88.0                    1
16    Adrian Houser   86.0                    2
17    Adrian Houser   86.0                    2
18    Adrian Houser   86.0                    2
19       Alex Reyes   97.0              

In [31]:
# Using OVER() with ORDER BY, partitioning by player_name - mimicking what ties would look like - with DENSE_RANK
sql_query = """
SELECT player_name, ROUND(release_speed), 
    DENSE_RANK() OVER(PARTITION BY player_name ORDER BY ROUND(release_speed) DESC) pitchers_speed_rank
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY player_name, pitchers_speed_rank
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  round  pitchers_speed_rank
0       A.J. Minter   97.0                    1
1       A.J. Minter   93.0                    2
2       A.J. Minter   92.0                    3
3   Adam Wainwright   91.0                    1
4   Adam Wainwright   91.0                    1
5   Adam Wainwright   91.0                    1
6   Adam Wainwright   91.0                    1
7   Adam Wainwright   88.0                    2
8   Adam Wainwright   88.0                    2
9   Adam Wainwright   86.0                    3
10  Adam Wainwright   85.0                    4
11  Adam Wainwright   82.0                    5
12  Adam Wainwright   76.0                    6
13  Adam Wainwright   75.0                    7
14  Adam Wainwright   75.0                    7
15    Adrian Houser   88.0                    1
16    Adrian Houser   86.0                    2
17    Adrian Houser   86.0                    2
18    Adrian Houser   86.0                    2
19       Alex Reyes   97.0              

In [ ]:
## Using a rank based on number of 

# https://leetcode.com/problems/rank-scores/

In [12]:
# Mimicking what ties would look like...
sql_query = """
SELECT player_name, ROUND(release_speed) AS rounded_speed
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND player_name LIKE '%Wainwright%'
ORDER BY rounded_speed DESC
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  rounded_speed
0   Adam Wainwright           91.0
1   Adam Wainwright           91.0
2   Adam Wainwright           91.0
3   Adam Wainwright           91.0
4   Adam Wainwright           88.0
5   Adam Wainwright           88.0
6   Adam Wainwright           86.0
7   Adam Wainwright           85.0
8   Adam Wainwright           82.0
9   Adam Wainwright           76.0
10  Adam Wainwright           75.0
11  Adam Wainwright           75.0


In [14]:
# Mimicking what ties would look like...
sql_query = """
SELECT player_name, ROUND(release_speed) AS rounded_speed
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND player_name LIKE '%Wainwright%'
ORDER BY rounded_speed DESC
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  rounded_speed
0   Adam Wainwright           91.0
1   Adam Wainwright           91.0
2   Adam Wainwright           91.0
3   Adam Wainwright           91.0
4   Adam Wainwright           88.0
5   Adam Wainwright           88.0
6   Adam Wainwright           86.0
7   Adam Wainwright           85.0
8   Adam Wainwright           82.0
9   Adam Wainwright           76.0
10  Adam Wainwright           75.0
11  Adam Wainwright           75.0


In [29]:
sql_query = """
SELECT player_name, release_speed, 
    RANK() OVER(PARTITION BY player_name ORDER BY release_speed DESC) pitchers_speed_rank
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY player_name, pitchers_speed_rank
LIMIT 20;
"""

In [ ]:
# From Leetcode
SELECT score, 
(SELECT COUNT(DISTINCT score) FROM scores WHERE score >= s.score)
 AS rank
FROM scores AS s
ORDER BY score DESC;

In [15]:
# Getting differences in consecutive rows
sql_query = """
SELECT player_name, ROUND(release_speed) AS rounded_speed
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
AND player_name LIKE '%Wainwright%'
ORDER BY rounded_speed DESC
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

        player_name  rounded_speed
0   Adam Wainwright           91.0
1   Adam Wainwright           91.0
2   Adam Wainwright           91.0
3   Adam Wainwright           91.0
4   Adam Wainwright           88.0
5   Adam Wainwright           88.0
6   Adam Wainwright           86.0
7   Adam Wainwright           85.0
8   Adam Wainwright           82.0
9   Adam Wainwright           76.0
10  Adam Wainwright           75.0
11  Adam Wainwright           75.0


## Using CASE - show Tatis as being behind, ahead, or equal in the count

In [60]:
# Example with case statement
sql_query = """
SELECT game_date, inning, balls, strikes, 
    CASE WHEN balls > strikes THEN 'ahead'
         WHEN balls < strikes THEN 'behind'
         ELSE 'equal' END AS count_status
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date  inning  balls  strikes count_status
0 2019-03-28     2.0    0.0      0.0        equal
1 2019-03-28     2.0    0.0      1.0       behind
2 2019-03-28     2.0    1.0      1.0        equal
3 2019-03-28     2.0    2.0      1.0        ahead
4 2019-03-28     4.0    0.0      0.0        equal
5 2019-03-28     7.0    0.0      0.0        equal
6 2019-03-28     7.0    0.0      1.0       behind
7 2019-03-28     7.0    0.0      2.0       behind
8 2019-03-28     7.0    1.0      2.0       behind
9 2019-03-29     2.0    0.0      0.0        equal


## Using CASE (without ELSE) - show Tatis as being behind, ahead, or equal in the count

In [64]:
# Example with case statement
sql_query = """
SELECT game_date, inning, balls, strikes, 
    (CASE WHEN balls > strikes THEN 'ahead'
         WHEN balls < strikes THEN 'behind'
         END) AS count_status
FROM statcast
JOIN player_id
ON statcast.batter=player_id.key_mlbam
WHERE key_mlbam=
    (SELECT key_mlbam FROM player_id
    WHERE name_last='tatis'
    AND name_first='fernando')
ORDER BY game_date, inning, balls, strikes
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date  inning  balls  strikes count_status
0 2019-03-28     2.0    0.0      0.0         None
1 2019-03-28     2.0    0.0      1.0       behind
2 2019-03-28     2.0    1.0      1.0         None
3 2019-03-28     2.0    2.0      1.0        ahead
4 2019-03-28     4.0    0.0      0.0         None
5 2019-03-28     7.0    0.0      0.0         None
6 2019-03-28     7.0    0.0      1.0       behind
7 2019-03-28     7.0    0.0      2.0       behind
8 2019-03-28     7.0    1.0      2.0       behind
9 2019-03-29     2.0    0.0      0.0         None


In [ ]:
# SQL practice - pick max speed for each pitcher



## Rank the top 3 pitchers who have thrown the most pitches per day

In [12]:
# Will eventually use OVER() with ORDER BY, partitioning by game_date

# But start with how things look using group by

sql_query = """
SELECT game_date, player_name, COUNT(release_speed) AS n_of_pitches
FROM statcast_simple
GROUP BY game_date, player_name
ORDER BY game_date, n_of_pitches DESC
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date        player_name  n_of_pitches
0 2018-07-07       Shawn Kelley             1
1 2018-07-09         Brad Brach             1
2 2018-07-15     Marcus Stroman            95
3 2018-07-15      Brian Johnson            84
4 2018-07-15  Jeremy Hellickson            73
5 2018-07-15         Mike Minor            68
6 2018-07-15        Mike Wright            64
7 2018-07-15       Corey Oswalt            59
8 2018-07-15      Miguel Castro            59
9 2018-07-15       Adam Kolarek            30


In [8]:
# Try adding in OVER() with ORDER BY, partitioning by game_date

# Problem is that I can't use the alias in the WHERE...
sql_query = """
SELECT game_date, player_name, COUNT(release_speed) AS n_pitches,
    RANK() OVER(PARTITION BY game_date ORDER BY COUNT(release_speed) DESC) AS n_pitches_rank
FROM statcast_simple
GROUP BY game_date, player_name
ORDER BY game_date
LIMIT 20;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    game_date          player_name  n_pitches  n_pitches_rank
0  2018-07-07         Shawn Kelley          1               1
1  2018-07-09           Brad Brach          1               1
2  2018-07-15       Marcus Stroman         95               1
3  2018-07-15        Brian Johnson         84               2
4  2018-07-15    Jeremy Hellickson         73               3
5  2018-07-15           Mike Minor         68               4
6  2018-07-15          Mike Wright         64               5
7  2018-07-15        Miguel Castro         59               6
8  2018-07-15         Corey Oswalt         59               6
9  2018-07-15        Alan Busenitz         30               8
10 2018-07-15         Adam Kolarek         30               8
11 2018-07-15        Matt Andriese         29              10
12 2018-07-15        Tyler Bashlor         28              11
13 2018-07-15  Trevor Hildenberger         27              12
14 2018-07-15       Ryan Yarbrough         25              13
15 2018-

In [63]:
# Use a CTE so I can use WHERE -

sql_query = """
WITH windowed_table AS 
    (SELECT game_date, player_name, COUNT(release_speed) AS n_pitches,
        DENSE_RANK() OVER(PARTITION BY game_date ORDER BY COUNT(release_speed) DESC) AS n_pitches_rank
    FROM statcast_simple
    GROUP BY game_date, player_name
    ORDER BY game_date)
SELECT * FROM windowed_table
WHERE windowed_table.n_pitches_rank < 4
LIMIT 50;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    game_date           player_name  n_pitches  n_pitches_rank
0  2018-07-07          Shawn Kelley          1               1
1  2018-07-09            Brad Brach          1               1
2  2018-07-15        Marcus Stroman         95               1
3  2018-07-15         Brian Johnson         84               2
4  2018-07-15     Jeremy Hellickson         73               3
5  2018-07-19        Kyle Hendricks        113               1
6  2018-07-19       Carlos Martinez         90               2
7  2018-07-19          Steve Cishek         23               3
8  2018-07-20           Danny Duffy        114               1
9  2018-07-20           Tyler Mahle        111               2
10 2018-07-20        Anibal Sanchez        111               2
11 2018-07-20         Jack Flaherty        110               3
12 2018-07-21            Lance Lynn        118               1
13 2018-07-21        Marcus Stroman        107               2
14 2018-07-21           Dylan Covey        105         

In [42]:
# Withot a window function

sql_query = """
SELECT game_date, player_name, COUNT(release_speed) AS n_of_pitches
FROM statcast_simple
GROUP BY game_date, player_name
ORDER BY game_date, n_of_pitches DESC
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date        player_name  n_of_pitches
0 2018-07-07       Shawn Kelley             1
1 2018-07-09         Brad Brach             1
2 2018-07-15     Marcus Stroman            95
3 2018-07-15      Brian Johnson            84
4 2018-07-15  Jeremy Hellickson            73
5 2018-07-15         Mike Minor            68
6 2018-07-15        Mike Wright            64
7 2018-07-15       Corey Oswalt            59
8 2018-07-15      Miguel Castro            59
9 2018-07-15       Adam Kolarek            30


In [ ]:
sql_query = """
SELECT game_date, player_name, COUNT(release_speed) AS n_of_pitches
FROM statcast_simple
GROUP BY game_date, player_name
ORDER BY game_date, n_of_pitches DESC
OFFSET LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

In [27]:
# Try using SELECT DISTINCT

# can't use nested aggregate function

sql_query = """
    SELECT game_date, 
           player_name, 
           COUNT(release_speed) AS n_pitches, 
           COUNT (DISTINCT COUNT(release_speed)) AS rank_n_pitches
     FROM statcast_simple
     GROUP BY game_date, player_name;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

DatabaseError: Execution failed on sql '
    SELECT game_date, 
           player_name, 
           COUNT(release_speed) AS n_pitches, 
           COUNT (DISTINCT COUNT(release_speed)) AS rank_n_pitches
     FROM statcast_simple
     GROUP BY game_date, player_name;
': aggregate function calls cannot be nested
LINE 5:            COUNT (DISTINCT COUNT(release_speed)) AS rank_n_p...
                                   ^


In [59]:
# Using CTE and then SELECT COUNT DISTINCT - this doesn't have date - probably window function is best


sql_query = """
WITH pitch_counts_by_date AS
    (SELECT game_date, player_name, COUNT(release_speed) AS n_of_pitches
    FROM statcast_simple
    GROUP BY game_date, player_name
    ORDER BY game_date, n_of_pitches DESC
    LIMIT 10)

SELECT game_date, 
       player_name,
       n_of_pitches,
       (SELECT COUNT(DISTINCT n_of_pitches) 
        FROM pitch_counts_by_date 
        WHERE n_of_pitches >= pc.n_of_pitches)
FROM pitch_counts_by_date AS pc
ORDER BY game_date ASC,
         n_of_pitches DESC
;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   game_date        player_name  n_of_pitches  count
0 2018-07-07       Shawn Kelley             1      8
1 2018-07-09         Brad Brach             1      8
2 2018-07-15     Marcus Stroman            95      1
3 2018-07-15      Brian Johnson            84      2
4 2018-07-15  Jeremy Hellickson            73      3
5 2018-07-15         Mike Minor            68      4
6 2018-07-15        Mike Wright            64      5
7 2018-07-15       Corey Oswalt            59      6
8 2018-07-15      Miguel Castro            59      6
9 2018-07-15       Adam Kolarek            30      7


## Simple review of CTE tables, UNION

In [9]:
# CTE
sql_query = """
WITH
    first_five AS
        (SELECT *
        FROM player_id
        LIMIT 5),
    second_five AS
        (SELECT *
        FROM player_id
        OFFSET 5 LIMIT 5)

SELECT * FROM first_five
UNION
SELECT * FROM second_five;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   index name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0      9     adams     chance     664856  adamc002  adamsch01          18135   
1      8     adams     austin     542866  adama001  adamsau01           7411   
2      7     adams     austin     613534  adama002  adamsau02          13801   
3      2     abreu       jose     547989  abrej003  abreujo02          15676   
4      0      abad   fernando     472551  abadf001   abadfe01           4994   
5      6    adames      willy     642715  adamw002  adamewi01          15986   
6      5    adames  cristhian     542436  adamc001  adamecr01           6013   
7      1     abreu      bryan     650556  abreb002  abreubr01          16609   
8      4      adam      jason     592094  adamj002   adamja01          11861   
9      3     acuna     ronald     660670  acunr001  acunaro01          18401   

   mlb_played_first  mlb_played_last  
0            2018.0           2019.0  
1            2014.0           2019.0  
2 

## Using simple table for complicated queries

In [13]:
sql_query = """
SELECT *
FROM statcast_simple;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      index pitch_type  game_date  release_speed home_team away_team  \
0         0         FT 2017-06-09           89.7        TB       OAK   
1         1         FC 2017-06-09           83.8        TB       OAK   
2         2         CH 2017-06-09           82.6        TB       OAK   
3         3         FT 2017-06-09           90.0        TB       OAK   
4         4         FT 2017-06-09           89.9        TB       OAK   
...     ...        ...        ...            ...       ...       ...   
9995   9995         FF 2017-06-07           92.2       DET       LAA   
9996   9996         FF 2017-06-07           91.9       DET       LAA   
9997   9997         FF 2017-06-07           92.1       DET       LAA   
9998   9998         FF 2017-06-07           93.0       DET       LAA   
9999   9999         SL 2017-06-07           87.3       DET       LAA   

        batter   pitcher  balls  strikes  home_score  away_score  
0     446334.0  594943.0    1.0      0.0        13.0         0.0  
1

## Using a subquery - show the number of pitches that are greater than the avg release speed

In [12]:
sql_query = """
SELECT *
FROM statcast_simple
WHERE release_speed > (SELECT AVG(release_speed)
        FROM statcast_simple);
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      index pitch_type  game_date  release_speed home_team away_team  \
0         0         FT 2017-06-09           89.7        TB       OAK   
1         3         FT 2017-06-09           90.0        TB       OAK   
2         4         FT 2017-06-09           89.9        TB       OAK   
3         5         FT 2017-06-09           89.8        TB       OAK   
4         6         FT 2017-06-09           89.9        TB       OAK   
...     ...        ...        ...            ...       ...       ...   
5832   9994         FF 2017-06-07           93.5       DET       LAA   
5833   9995         FF 2017-06-07           92.2       DET       LAA   
5834   9996         FF 2017-06-07           91.9       DET       LAA   
5835   9997         FF 2017-06-07           92.1       DET       LAA   
5836   9998         FF 2017-06-07           93.0       DET       LAA   

        batter   pitcher  balls  strikes  home_score  away_score  
0     446334.0  594943.0    1.0      0.0        13.0         0.0  
1

## Using a self-join - find at-bats where a pitcher was a batter

In [8]:
# I think this is wrong
sql_query = """
SELECT *
FROM statcast_simple ss1, statcast_simple ss2
WHERE ss1.batter=ss2.pitcher;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

KeyboardInterrupt: 

In [11]:
sql_query = """
SELECT *
FROM statcast_simple AS ss1
JOIN statcast_simple AS ss2
ON ss1.home_team=ss2.away_team;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

KeyboardInterrupt: 

In [13]:
sql_query = """
SELECT *
FROM statcast_simple ss1
WHERE ss1.batter IN
    (SELECT DISTINCT(ss2.pitcher)
    FROM statcast_simple ss2);
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

     index pitch_type  game_date  release_speed    batter   pitcher  balls  \
0      118         FT 2017-06-14           88.9  607237.0  468504.0    0.0   
1      127         SL 2017-06-14           80.2  468504.0  607237.0    1.0   
2      128         FF 2017-06-14           91.2  468504.0  607237.0    1.0   
3      129         FF 2017-06-14           90.4  468504.0  607237.0    1.0   
4      130         SL 2017-06-14           78.2  468504.0  607237.0    0.0   
..     ...        ...        ...            ...       ...       ...    ...   
287   9826         SL 2017-06-11           85.6  502212.0  453286.0    2.0   
288   9827         FF 2017-06-11           95.7  502212.0  453286.0    2.0   
289   9828         FF 2017-06-11           94.3  502212.0  453286.0    1.0   
290   9829         FF 2017-06-11           94.3  502212.0  453286.0    0.0   
291   9830         FF 2017-06-11           93.6  502212.0  453286.0    0.0   

     strikes  home_score  away_score  
0        0.0         0.0

In [11]:
sql_query = """
SELECT * FROM player_id
WHERE key_mlbam IN
    (SELECT DISTINCT(ss1.batter)
    FROM statcast_simple ss1, statcast_simple ss2
    WHERE ss1.batter=ss2.pitcher);
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    index     name_last name_first  key_mlbam key_retro  key_bbref  \
0      84        arroyo    bronson     276520  arrob001  arroybr01   
1     168  bibens-dirkx     austin     502212  bibea001  bibenau01   
2     176         blach         ty     621389  blact002  blachty01   
3     319        chacin    jhoulys     468504  chacj001  chacijh01   
4     325      chatwood      tyler     543022  chatt001  chatwty01   
5     350          cole     gerrit     543037  coleg001   colege01   
6     378        cotton     jharel     605194  cottj001  cottojh01   
7     432        degrom      jacob     594798  degrj001  degroja01   
8     460        dickey      r. a.     285079  dickr001  dicker.01   
9     540       feldman      scott     444857  felds001  feldmsc01   
10    567   foltynewicz       mike     592314  foltm001  foltymi01   
11    584      freeland       kyle     607536  freek001  freelky01   
12    630       garrett       amir     607237  garra001  garream01   
13    650       gime

In [ ]:
sql_query = """
SELECT *
FROM statcast_simple
WHERE release_speed > (SELECT AVG(release_speed)
        FROM statcast_simple);
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

## Get the months with the highest avg release speeds

In [14]:
sql_query = """
SELECT * FROM statcast_simple
LIMIT 10
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   index pitch_type  game_date  release_speed home_team away_team    batter  \
0      0         FT 2017-06-09           89.7        TB       OAK  446334.0   
1      1         FC 2017-06-09           83.8        TB       OAK  446334.0   
2      2         CH 2017-06-09           82.6        TB       OAK  572816.0   
3      3         FT 2017-06-09           90.0        TB       OAK  605480.0   
4      4         FT 2017-06-09           89.9        TB       OAK  605480.0   
5      5         FT 2017-06-09           89.8        TB       OAK  605480.0   
6      6         FT 2017-06-09           89.9        TB       OAK  605480.0   
7      7         FT 2017-06-09           90.2        TB       OAK  491696.0   
8      8         FT 2017-06-09           89.8        TB       OAK  491696.0   
9      9         FC 2017-06-09           84.9        TB       OAK  491696.0   

    pitcher  balls  strikes  home_score  away_score  
0  594943.0    1.0      0.0        13.0         0.0  
1  594943.0    0.0    

In [19]:
# Extract month
sql_query = """
SELECT EXTRACT(MONTH FROM game_date) AS month
FROM statcast
LIMIT 10
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   month
0    6.0
1    6.0
2    6.0
3    6.0
4    6.0
5    6.0
6    6.0
7    6.0
8    6.0
9    6.0


In [20]:
# Group by average release speeds for each month - month alias can be used in group by?

sql_query = """
SELECT EXTRACT(MONTH FROM game_date) AS month, AVG(release_speed) AS avg_release_speed
FROM statcast
GROUP BY month
ORDER BY avg_release_speed;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   month  avg_release_speed
0    4.0          88.430430
1    5.0          88.544273
2    3.0          88.563039
3    8.0          88.643873
4    6.0          88.733679
5    7.0          88.741111
6   10.0          88.758476
7    9.0          88.769983


In [25]:
# Group by average release speeds for each month - check if batter alias can be used in group by???

sql_query = """
SELECT batter AS b_alias, AVG(release_speed) AS avg_release_speed
FROM statcast_simple
GROUP BY b_alias;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      b_alias  avg_release_speed
0    429666.0          89.565789
1    592663.0          90.721622
2    519076.0          83.000000
3    518792.0          92.524590
4    547957.0          91.175000
..        ...                ...
394  446263.0          87.417143
395  459431.0          85.225000
396  502100.0          89.935714
397  501981.0          86.831250
398  471865.0          89.106061

[399 rows x 2 columns]


In [27]:
# Group by average release speeds for each month - check if batter alias can be used in group by???
sql_query = """
SELECT CONCAT(away_team, 'vs', home_team) AS matchup, 
    AVG(release_speed) AS avg_release_speed
FROM statcast_simple
GROUP BY matchup
ORDER BY avg_release_speed DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

     matchup  avg_release_speed
0   LAAvsDET          91.747516
1   MIAvsPIT          90.760225
2   WSHvsLAD          90.221687
3   BOSvsNYY          90.185304
4   PITvsBAL          90.143766
5   COLvsCHC          89.763666
6   NYMvsTEX          89.615331
7    CWSvsTB          89.182215
8   MILvsARI          89.129781
9   TORvsOAK          88.965766
10   MINvsSF          88.758824
11  LAAvsHOU          88.742424
12  MIAvsCHC          88.712420
13  DETvsBOS          88.652454
14   HOUvsKC          88.554582
15   SDvsARI          88.446586
16   SFvsMIL          88.399389
17  NYMvsATL          88.272425
18  MINvsSEA          88.218233
19  BALvsWSH          87.939209
20  CWSvsCLE          87.785357
21    KCvsSD          87.659220
22  BALvsNYY          87.307143
23  CINvsLAD          87.162951
24  PHIvsATL          86.503352
25  STLvsCIN          86.233136
26   OAKvsTB          85.463684


In [52]:
# Group by average release speeds for each month - check if batter alias can be used in group by???
sql_query = """
SELECT away_team AS a_alias,
    AVG(release_speed) AS avg_release_speed
FROM statcast_simple
GROUP BY a_alias
ORDER BY avg_release_speed DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   a_alias  avg_release_speed
0      LAA          90.305654
1      WSH          90.221687
2      BOS          90.185304
3      PIT          90.143766
4      MIA          90.072513
5      COL          89.763666
6      MIL          89.129781
7      TOR          88.965766
8      NYM          88.927891
9      DET          88.652454
10     HOU          88.554582
11     CWS          88.505536
12      SD          88.446586
13      SF          88.399389
14     MIN          88.376892
15      KC          87.659220
16     BAL          87.630147
17     CIN          87.162951
18     PHI          86.503352
19     STL          86.233136
20     OAK          85.463684


In [28]:
sql_query = """
SELECT *
FROM statcast_simple;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      index pitch_type  game_date  release_speed home_team away_team  \
0         0         FT 2017-06-09           89.7        TB       OAK   
1         1         FC 2017-06-09           83.8        TB       OAK   
2         2         CH 2017-06-09           82.6        TB       OAK   
3         3         FT 2017-06-09           90.0        TB       OAK   
4         4         FT 2017-06-09           89.9        TB       OAK   
...     ...        ...        ...            ...       ...       ...   
9995   9995         FF 2017-06-07           92.2       DET       LAA   
9996   9996         FF 2017-06-07           91.9       DET       LAA   
9997   9997         FF 2017-06-07           92.1       DET       LAA   
9998   9998         FF 2017-06-07           93.0       DET       LAA   
9999   9999         SL 2017-06-07           87.3       DET       LAA   

        batter   pitcher  balls  strikes  home_score  away_score  
0     446334.0  594943.0    1.0      0.0        13.0         0.0  
1

In [ ]:
# Group by average release speeds for each month - check if batter alias can be used in group by???  
# --- check if it's allowed in window function


sql_query = """
SELECT CONCAT(away_team, 'vs', home_team) AS matchup, 
    AVG(release_speed) AS avg_release_speed
FROM statcast_simple
GROUP BY matchup
ORDER BY avg_release_speed DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

## Rank the top 3 away home run hitters for each team by month for 2019

In [ ]:
team | month | player | no_home_runs | team_rank

In [86]:
# extract month and year
sql_query = """
SELECT 
    EXTRACT(MONTH FROM game_date) AS month, 
    batter, 
    away_team
FROM statcast
WHERE events='home_run'
AND EXTRACT(YEAR FROM game_date)=2019
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   month    batter away_team
0    9.0  656371.0       MIA
1    9.0  456078.0       DET
2    9.0  408234.0       DET
3    9.0  543543.0       MIA
4    9.0  445988.0       MIA


In [111]:
# without using window function

sql_query = """
SELECT 
    EXTRACT(MONTH FROM game_date) AS month, 
    batter, 
    COUNT(events) AS no_home_runs
FROM statcast
WHERE events='home_run'
AND inning_topbot='Top'
AND game_date BETWEEN '2019-04-01' AND '2019-05-31'
GROUP BY month, batter
ORDER BY no_home_runs DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

     month    batter  no_home_runs
0      4.0  596129.0             7
1      4.0  572228.0             7
2      5.0  605137.0             7
3      4.0  596142.0             6
4      5.0  500871.0             6
..     ...       ...           ...
519    5.0  501896.0             1
520    4.0  465041.0             1
521    5.0  666971.0             1
522    5.0  502110.0             1
523    5.0  624428.0             1

[524 rows x 3 columns]


In [117]:
# window function by game_date

sql_query = """
SELECT 
    EXTRACT(MONTH FROM game_date) AS month, 
    batter, 
    COUNT(events) OVER(PARTITION BY batter, EXTRACT(MONTH FROM game_date)) AS no_home_runs
FROM statcast
WHERE events='home_run'
AND game_date BETWEEN '2019-04-01' AND '2019-05-31';
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

      month    batter  no_home_runs
0       4.0  405395.0             4
1       4.0  405395.0             4
2       4.0  405395.0             4
3       4.0  405395.0             4
4       5.0  405395.0             5
...     ...       ...           ...
2141    5.0  669222.0             4
2142    5.0  669374.0             4
2143    5.0  669374.0             4
2144    5.0  669374.0             4
2145    5.0  669374.0             4

[2146 rows x 3 columns]


In [127]:
# Window function - showing all ranks

sql_query = """
SELECT 
    away_team,
    EXTRACT(MONTH FROM game_date) AS month, 
    batter,
    COUNT(events),
    RANK() OVER(PARTITION BY away_team, EXTRACT(MONTH FROM game_date) ORDER BY COUNT(events) DESC) AS team_hr_rank
FROM statcast
WHERE events='home_run'
AND inning_topbot='Top'
AND game_date BETWEEN '2019-04-01' AND '2019-05-31'
GROUP BY away_team, month, batter
HAVING away_team='SD';
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   away_team  month    batter  count  team_hr_rank
0         SD    4.0  614177.0      5             1
1         SD    4.0  543333.0      4             2
2         SD    4.0  592669.0      4             2
3         SD    4.0  665487.0      4             2
4         SD    4.0  592518.0      3             5
5         SD    4.0  595978.0      2             6
6         SD    4.0  622534.0      2             6
7         SD    4.0  594824.0      1             8
8         SD    4.0  571976.0      1             8
9         SD    5.0  571976.0      5             1
10        SD    5.0  592669.0      4             2
11        SD    5.0  595978.0      2             3
12        SD    5.0  614177.0      2             3
13        SD    5.0  435079.0      2             3
14        SD    5.0  543333.0      2             3
15        SD    5.0  592518.0      2             3
16        SD    5.0  594824.0      1             8


In [132]:
# Window function - limiting to top 3

sql_query = """
SELECT *
FROM
    (SELECT 
        away_team,
        EXTRACT(MONTH FROM game_date) AS month, 
        batter,
        COUNT(events),
        RANK() OVER(PARTITION BY away_team, EXTRACT(MONTH FROM game_date) ORDER BY COUNT(events) DESC) AS team_hr_rank
    FROM statcast
    WHERE events='home_run'
    AND inning_topbot='Top'
    AND game_date BETWEEN '2019-04-01' AND '2019-05-31'
    GROUP BY away_team, month, batter
    HAVING away_team='SD')
AS tmp_table
WHERE team_hr_rank <=3;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   away_team  month    batter  count  team_hr_rank
0         SD    4.0  614177.0      5             1
1         SD    4.0  543333.0      4             2
2         SD    4.0  592669.0      4             2
3         SD    4.0  665487.0      4             2
4         SD    5.0  571976.0      5             1
5         SD    5.0  592669.0      4             2
6         SD    5.0  595978.0      2             3
7         SD    5.0  614177.0      2             3
8         SD    5.0  435079.0      2             3
9         SD    5.0  543333.0      2             3
10        SD    5.0  592518.0      2             3


In [133]:
# Window function - limiting to top 3

sql_query = """
SELECT *
FROM
    (SELECT 
        away_team,
        EXTRACT(MONTH FROM game_date) AS month, 
        batter,
        COUNT(events),
        RANK() OVER(PARTITION BY away_team, EXTRACT(MONTH FROM game_date) ORDER BY COUNT(events) DESC) AS team_hr_rank
    FROM statcast
    WHERE events='home_run'
    AND inning_topbot='Top'
    AND EXTRACT(YEAR FROM game_date)=2019
    GROUP BY away_team, month, batter
    HAVING away_team='SD')
AS tmp_table
WHERE team_hr_rank <=3;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   away_team  month    batter  count  team_hr_rank
0         SD    4.0  614177.0      5             1
1         SD    4.0  543333.0      4             2
2         SD    4.0  592669.0      4             2
3         SD    4.0  665487.0      4             2
4         SD    5.0  571976.0      5             1
5         SD    5.0  592669.0      4             2
6         SD    5.0  543333.0      2             3
7         SD    5.0  592518.0      2             3
8         SD    5.0  435079.0      2             3
9         SD    5.0  595978.0      2             3
10        SD    5.0  614177.0      2             3
11        SD    6.0  592669.0      6             1
12        SD    6.0  592518.0      6             1
13        SD    6.0  665487.0      2             3
14        SD    6.0  614177.0      2             3
15        SD    7.0  665487.0      4             1
16        SD    7.0  622534.0      3             2
17        SD    7.0  592669.0      3             2
18        SD    8.0  642336.0  

## What teams have done best against the shift?

In [ ]:
| rank | team | OBP overall | OBP normal |  OBP vs shift 

In [136]:
# All queries
sql_query = """
SELECT * FROM statcast LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   level_0  index pitch_type  game_date  release_speed  release_pos_x  \
0      611    959         FS 2019-09-29           82.6        -2.8489   
1      612    976         FF 2019-09-29           94.4        -2.6751   
2      613    991         FF 2019-09-29           94.1        -2.5993   
3      614   1006         FS 2019-09-29           81.0        -2.9759   
4      615   1021         CU 2019-09-29           74.3        -3.0207   

   release_pos_z  player_name    batter   pitcher  ... home_score away_score  \
0         6.4584  Wilmer Font  643275.0  521655.0  ...        6.0        1.0   
1         6.5472  Wilmer Font  643275.0  521655.0  ...        6.0        1.0   
2         6.5978  Wilmer Font  643275.0  521655.0  ...        6.0        1.0   
3         6.4140  Wilmer Font  643275.0  521655.0  ...        6.0        1.0   
4         6.4187  Wilmer Font  572287.0  521655.0  ...        6.0        1.0   

  bat_score fld_score post_away_score post_home_score  post_bat_score  \
0      

In [139]:
# Calculate OBP against shift for each team

# Use SD to narrow down query
sql_query = """
SELECT batter, home_team, events, if_fielding_alignment
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND home_team='SD'
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


     batter home_team              events if_fielding_alignment
0  471865.0        SD           field_out         Infield shift
1  448602.0        SD           strikeout         Infield shift
2  448602.0        SD  caught_stealing_2b         Infield shift
3  471865.0        SD           strikeout         Infield shift
4  572114.0        SD           strikeout         Infield shift


In [145]:
# Calculate OBP against shift for each team

# Use SD to narrow down query and make total counts of OB events
sql_query = """

SELECT 
(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND events IN ('single', 'double', 'triple', 'home_run')
AND home_team='SD') AS hit_shift_PAs,
(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND home_team='SD') AS total_shift_PAs;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


   hit_shift_pas  total_shift_pas
0            445             2163


In [146]:
# Calculate OBP against shift for each team

# Use SD to narrow down query and make total counts of OB events
sql_query = """

SELECT 
(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND events IN ('single', 'double', 'triple', 'home_run')
AND home_team='SD') AS hit_shift_PAs,
(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND home_team='SD') AS total_shift_PAs;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


   hit_shift_pas  total_shift_pas
0            445             2163


In [10]:
# Calculate OBP against shift for each team

# Use SD to narrow down query and make total counts of OB events
sql_query = """

SELECT 

(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND events IN ('single', 'double', 'triple', 'home_run')
AND home_team='SD'
AND inning_topbot='Bot') AS hit_shift_PAs,

(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND home_team='SD'
AND inning_topbot='Bot') AS total_shift_PAs,

(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND events IN ('single', 'double', 'triple', 'home_run')
AND home_team='SD'
AND inning_topbot='Bot')::decimal / 
(SELECT COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND home_team='SD'
AND inning_topbot='Bot') AS obp_shift;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


   hit_shift_pas  total_shift_pas  obp_shift
0            200             1004   0.199203


In [150]:
# Calculate OBP against shift for each team

# Use group by to get each team
sql_query = """
SELECT home_team, COUNT(batter)
FROM statcast
WHERE if_fielding_alignment='Infield shift'
AND events IS NOT NULL
AND events IN ('single', 'double', 'triple', 'home_run')
AND inning_topbot='Bot'
GROUP BY home_team;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


   home_team  count
0        ARI    276
1        ATL    375
2        BAL    402
3        BOS    443
4        CHC    503
5        CIN    321
6        CLE    463
7        COL    369
8        CWS    221
9        DET    319
10       HOU    364
11        KC    374
12       LAA    447
13       LAD    499
14       MIA    189
15       MIL    257
16       MIN    457
17       NYM    392
18       NYY    433
19       OAK    420
20       PHI    344
21       PIT    197
22        SD    200
23       SEA    358
24        SF    272
25       STL    348
26        TB    380
27       TEX    474
28       TOR    521
29       WSH    304


In [157]:
# Calculate OBP against shift for each team

# Use SD to narrow down query and make total counts of OB events
sql_query = """

SELECT 
     hit_shift_PAs_table.home_team, 
     hit_shift_PAs_table.count, 
     total_shift_PAs_table.count,
     (hit_shift_PAs_table.count::decimal / total_shift_PAs_table.count) AS obp_shift
FROM
    (SELECT home_team, COUNT(batter)
    FROM statcast
    WHERE if_fielding_alignment='Infield shift'
    AND events IS NOT NULL
    AND events IN ('single', 'double', 'triple', 'home_run')
    AND inning_topbot='Bot'
    GROUP BY home_team) AS hit_shift_PAs_table
JOIN
    (SELECT home_team, COUNT(batter)
    FROM statcast
    WHERE if_fielding_alignment='Infield shift'
    AND events IS NOT NULL
    AND inning_topbot='Bot'
    GROUP BY home_team) AS total_shift_PAs_table
ON
   hit_shift_PAs_table.home_team=total_shift_PAs_table.home_team
ORDER BY obp_shift DESC;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   home_team  count  count  obp_shift
0        COL    369   1310   0.281679
1        WSH    304   1228   0.247557
2        HOU    364   1504   0.242021
3        CHC    503   2107   0.238728
4        ATL    375   1578   0.237643
5        BOS    443   1894   0.233897
6         KC    374   1607   0.232732
7        DET    319   1380   0.231159
8        STL    348   1533   0.227006
9        CIN    321   1417   0.226535
10       MIN    457   2036   0.224460
11        SF    272   1226   0.221860
12       LAD    499   2250   0.221778
13       NYY    433   1955   0.221483
14       MIL    257   1162   0.221170
15       PHI    344   1571   0.218969
16       OAK    420   1923   0.218409
17       CLE    463   2167   0.213659
18       CWS    221   1046   0.211281
19       MIA    189    895   0.211173
20        TB    380   1807   0.210293
21       TOR    521   2520   0.206746
22       BAL    402   1960   0.205102
23       PIT    197    967   0.203723
24       ARI    276   1357   0.203390
25       TEX

## Build on above, but use CASE to calculate based on away or home to get stats per team, regardless of its home or away

In [17]:
# Calculate OBP against shift for each team

# Use group by to get each team, use CASE to count for away or home
sql_query = """
SELECT * FROM statcast_simple
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


   index pitch_type  game_date  release_speed home_team away_team    batter  \
0      0         FT 2017-06-09           89.7        TB       OAK  446334.0   
1      1         FC 2017-06-09           83.8        TB       OAK  446334.0   
2      2         CH 2017-06-09           82.6        TB       OAK  572816.0   
3      3         FT 2017-06-09           90.0        TB       OAK  605480.0   
4      4         FT 2017-06-09           89.9        TB       OAK  605480.0   
5      5         FT 2017-06-09           89.8        TB       OAK  605480.0   
6      6         FT 2017-06-09           89.9        TB       OAK  605480.0   
7      7         FT 2017-06-09           90.2        TB       OAK  491696.0   
8      8         FT 2017-06-09           89.8        TB       OAK  491696.0   
9      9         FC 2017-06-09           84.9        TB       OAK  491696.0   

    pitcher  balls  strikes  home_score  away_score  
0  594943.0    1.0      0.0        13.0         0.0  
1  594943.0    0.0    

In [18]:
# Calculate OBP against shift for each team

# Use group by to get each team, use CASE to count for away or home
sql_query = """
SELECT home_team
FROM statcast
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)


  home_team
0        SD
1        SD
2        SD
3        SD
4        SD


In [35]:
# Calculate OBP against shift for each team

# Use group by to get each team, use CASE to count for away or home
sql_query = """

SELECT ob_table.team_name, 
       ob_table.n_ob, 
       total_ab_table.n_total_ab,
       ob_table.n_ob::decimal/total_ab_table.n_total_ab AS teams_prop_ob
FROM
    (SELECT 
        CASE WHEN inning_topbot='Top' THEN away_team
             WHEN inning_topbot='Bot' THEN home_team
             END AS team_name,
        COUNT(batter) AS n_ob
    FROM statcast
    WHERE if_fielding_alignment='Infield shift'
    AND events IS NOT NULL
    AND events IN ('single', 'double', 'triple', 'home_run')
    GROUP BY team_name
    LIMIT 5) AS ob_table
JOIN
    (SELECT 
        CASE WHEN inning_topbot='Top' THEN away_team
             WHEN inning_topbot='Bot' THEN home_team
             END AS team_name,
        COUNT(batter) AS n_total_ab
    FROM statcast
    WHERE if_fielding_alignment='Infield shift'
    AND events IS NOT NULL
    GROUP BY team_name
    LIMIT 5) AS total_ab_table
ON
    ob_table.team_name=total_ab_table.team_name;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  team_name  n_ob  n_total_ab  teams_prop_ob
0       ARI   545        2586       0.210750
1       ATL   783        3280       0.238720
2       BAL   757        3727       0.203112
3       BOS   832        3782       0.219989
4       CHC   994        4428       0.224481


In [46]:
# Calculate OBP against shift for each team

# Use group by to get each team, use CASE to count for away or home
sql_query = """

SELECT 
    ob_events_table.team_name,
    ob_events_table.if_fielding_alignment,
    ob_events_table.n_ob,
    total_ab_table.n_ob,
    ob_events_table.n_ob::decimal/total_ab_table.n_ob AS obp
FROM
    (SELECT 
        CASE WHEN inning_topbot='Top' THEN away_team
             WHEN inning_topbot='Bot' THEN home_team
             END AS team_name,
        if_fielding_alignment,
        COUNT(batter) AS n_ob
    FROM statcast
    WHERE events IS NOT NULL
    AND events IN ('single', 'double', 'triple', 'home_run')
    GROUP BY team_name, if_fielding_alignment
    HAVING if_fielding_alignment IN ('Infield shift', 'Standard')) AS ob_events_table
JOIN
    (SELECT 
        CASE WHEN inning_topbot='Top' THEN away_team
             WHEN inning_topbot='Bot' THEN home_team
             END AS team_name,
        if_fielding_alignment,
        COUNT(batter) AS n_ob
    FROM statcast
    WHERE events IS NOT NULL
    GROUP BY team_name, if_fielding_alignment
    HAVING if_fielding_alignment IN ('Infield shift', 'Standard')) AS total_ab_table
ON
    ob_events_table.team_name=total_ab_table.team_name
    AND
    ob_events_table.if_fielding_alignment=total_ab_table.if_fielding_alignment;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   team_name if_fielding_alignment  n_ob   n_ob       obp
0        ARI         Infield shift   545   2586  0.210750
1        ARI              Standard  3244  14386  0.225497
2        ATL         Infield shift   783   3280  0.238720
3        ATL              Standard  3153  13437  0.234651
4        BAL         Infield shift   757   3727  0.203112
5        BAL              Standard  3172  13572  0.233716
6        BOS         Infield shift   832   3782  0.219989
7        BOS              Standard  3255  13418  0.242585
8        CHC         Infield shift   994   4428  0.224481
9        CHC              Standard  2787  12021  0.231844
10       CIN         Infield shift   654   2903  0.225284
11       CIN              Standard  3070  13655  0.224826
12       CLE         Infield shift   964   4515  0.213511
13       CLE              Standard  2971  12582  0.236131
14       COL         Infield shift   684   2724  0.251101
15       COL              Standard  3278  13734  0.238678
16       CWS  

## Find the batters that faced exactly three pitches

From Leetcode  - write a SQL query to find all numbers that appear **at least** three times consecutively
https://leetcode.com/problems/consecutive-numbers/

In [49]:
# Explore
sql_query = """
SELECT * FROM statcast_simple
LIMIT 8;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   index pitch_type  game_date  release_speed home_team away_team    batter  \
0      0         FT 2017-06-09           89.7        TB       OAK  446334.0   
1      1         FC 2017-06-09           83.8        TB       OAK  446334.0   
2      2         CH 2017-06-09           82.6        TB       OAK  572816.0   
3      3         FT 2017-06-09           90.0        TB       OAK  605480.0   
4      4         FT 2017-06-09           89.9        TB       OAK  605480.0   
5      5         FT 2017-06-09           89.8        TB       OAK  605480.0   
6      6         FT 2017-06-09           89.9        TB       OAK  605480.0   
7      7         FT 2017-06-09           90.2        TB       OAK  491696.0   

    pitcher  balls  strikes  home_score  away_score  
0  594943.0    1.0      0.0        13.0         0.0  
1  594943.0    0.0      0.0        13.0         0.0  
2  594943.0    0.0      0.0        10.0         0.0  
3  594943.0    1.0      2.0        10.0         0.0  
4  594943.0    0.0

In [123]:
# Get times that a batter appears in order
sql_query = """
SELECT batter FROM statcast_simple
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

     batter
0  622110.0
1  622110.0
2  622110.0
3  595281.0
4  519317.0
5  519317.0
6  519317.0
7  519317.0
8  519317.0
9  519317.0


In [64]:
# I'm not aware of an approach so I looked at solution

# Make multiple tables, then use SELECT DISTINCT and WHERE, using the ids/indexes

sql_query = """
WITH sc20 AS 
    (SELECT index, pitch_type, batter FROM statcast_simple
    LIMIT 20)
SELECT *
FROM sc20 sc_a;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

    index pitch_type    batter
0       0         FT  446334.0
1       1         FC  446334.0
2       2         CH  572816.0
3       3         FT  605480.0
4       4         FT  605480.0
5       5         FT  605480.0
6       6         FT  605480.0
7       7         FT  491696.0
8       8         FT  491696.0
9       9         FC  491696.0
10     10         FT  621002.0
11     11         FC  621002.0
12     12         FT  621002.0
13     13         FF  542921.0
14     14         FF  595144.0
15     15         KC  595144.0
16     16         FS  595144.0
17     17         FF  595144.0
18     18         FS  595144.0
19     19         FF  640461.0


In [63]:
# Make multiple tables, then and use WHERE for the ids/indexes/positions

sql_query = """
WITH sc20 AS 
    (SELECT index, pitch_type, batter FROM statcast_simple
    LIMIT 20)
    
SELECT sc_a.index, sc_b.index, sc_c.index, sc_a.batter, sc_b.batter, sc_c.batter
FROM
     sc20 sc_a,
     sc20 sc_b,
     sc20 sc_c
WHERE
    sc_a.index = sc_b.index-1
AND sc_b.index = sc_c.index-1   
AND sc_a.batter = sc_b.batter
AND sc_b.batter = sc_c.batter;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   index  index  index    batter    batter    batter
0      3      4      5  605480.0  605480.0  605480.0
1      4      5      6  605480.0  605480.0  605480.0
2      7      8      9  491696.0  491696.0  491696.0
3     10     11     12  621002.0  621002.0  621002.0
4     14     15     16  595144.0  595144.0  595144.0
5     15     16     17  595144.0  595144.0  595144.0
6     16     17     18  595144.0  595144.0  595144.0


In [125]:
# Make multiple tables, then and use WHERE for the ids/indexes/positions, then use SELECT DISTINCT, to output a list of numbers

sql_query = """
WITH sc20 AS 
    (SELECT index, pitch_type, batter FROM statcast_simple
    LIMIT 20)
    
SELECT DISTINCT sc_a.batter AS consecutive_numbers
FROM
     sc20 AS sc_a,
     sc20 AS sc_b,
     sc20 AS sc_c
WHERE
    sc_a.index = sc_b.index-1
AND sc_b.index = sc_c.index-1   
AND sc_a.batter = sc_b.batter
AND sc_b.batter = sc_c.batter;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   consecutive_numbers
0             519317.0
1             544369.0
2             592450.0
3             622110.0


## Determine the difference in speed from the previous pitch, without and with using lag/lead

In [6]:
# Using lag as window function
sql_query = """
SELECT *
FROM statcast_simple;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

       index pitch_type  game_date  release_speed home_team away_team  \
0          0         SL 2018-07-25           82.8        TB       NYY   
1          1         FT 2018-07-25           93.1        TB       NYY   
2          2         FF 2018-07-25           92.4        TB       NYY   
3          3         FF 2018-07-25           92.5        TB       NYY   
4          4         SL 2018-07-25           89.8        TB       NYY   
...      ...        ...        ...            ...       ...       ...   
24995  24995         CH 2018-07-15           88.2       MIN        TB   
24996  24996         FF 2018-07-15           95.7       MIN        TB   
24997  24997         CH 2018-07-15           90.9       MIN        TB   
24998  24998         SL 2018-07-15           86.6       MIN        TB   
24999  24999         FT 2018-07-15           94.5       MIN        TB   

         batter   pitcher      player_name  balls  strikes  home_score  \
0      622110.0  570666.0       Luis Cessa    1.0

In [17]:
# See if you can use index
sql_query = """
SELECT COUNT(DISTINCT(index))
FROM statcast_simple;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   count
0  25000


In [24]:
# Get previous speed without a window function
sql_query = """
SELECT index, pitch_type, release_speed,
    (SELECT release_speed
     FROM statcast_simple AS ss1
     WHERE ss1.index=statcast_simple.index-1) AS prev_pitch_speed,
     (release_speed - (SELECT release_speed
     FROM statcast_simple AS ss1
     WHERE ss1.index=statcast_simple.index-1)) AS diff_in_speed
FROM statcast_simple;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

       index pitch_type  release_speed  prev_pitch_speed  diff_in_speed
0          0         SL           82.8               NaN            NaN
1          1         FT           93.1              82.8           10.3
2          2         FF           92.4              93.1           -0.7
3          3         FF           92.5              92.4            0.1
4          4         SL           89.8              92.5           -2.7
...      ...        ...            ...               ...            ...
24995  24995         CH           88.2              87.8            0.4
24996  24996         FF           95.7              88.2            7.5
24997  24997         CH           90.9              95.7           -4.8
24998  24998         SL           86.6              90.9           -4.3
24999  24999         FT           94.5              86.6            7.9

[25000 rows x 5 columns]


In [29]:
# Using lag as window function
sql_query = """
SELECT 
    pitch_type, release_speed,
    LAG(release_speed, 1) OVER() AS prev_pitch_speed,
    release_speed - (LAG(release_speed, 1) OVER()) AS diff_in_speed
FROM statcast_simple
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  release_speed  prev_pitch_speed  diff_in_speed
0         SL           82.8               NaN            NaN
1         FT           93.1              82.8           10.3
2         FF           92.4              93.1           -0.7
3         FF           92.5              92.4            0.1
4         SL           89.8              92.5           -2.7
5         FF           99.0              89.8            9.2
6         FS           86.5              99.0          -12.5
7         SL           87.9              86.5            1.4
8         FF           97.0              87.9            9.1
9         FF           97.7              97.0            0.7


In [27]:
# Using lead as window function - look at 2 pitches ahead
sql_query = """
SELECT 
    pitch_type, release_speed,
    LEAD(release_speed, 2) OVER() AS two_pitches_ahead_speed,
    release_speed - (LEAD(release_speed, 2) OVER()) AS diff_in_future_speed
FROM statcast_simple
LIMIT 10;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

  pitch_type  release_speed  two_pitches_ahead_speed  diff_in_future_speed
0         SL           82.8                     92.4                  -9.6
1         FT           93.1                     92.5                   0.6
2         FF           92.4                     89.8                   2.6
3         FF           92.5                     99.0                  -6.5
4         SL           89.8                     86.5                   3.3
5         FF           99.0                     87.9                  11.1
6         FS           86.5                     97.0                 -10.5
7         SL           87.9                     97.7                  -9.8
8         FF           97.0                     98.1                  -1.1
9         FF           97.7                     97.8                  -0.1


## -template for next query-

In [112]:
# without using window function

sql_query = """
SELECT 
    EXTRACT(MONTH FROM game_date) AS month, 
    batter, 
    SUM(events) RANK() OVER(PARTITION BY month) AS no_home_runs
FROM statcast
WHERE events='home_run'
AND inning_topbot='Top'
AND game_date BETWEEN '2019-04-01' AND '2019-05-31';
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

DatabaseError: Execution failed on sql '
SELECT 
    EXTRACT(MONTH FROM game_date) AS month, 
    batter, 
    SUM(events) RANK() OVER(PARTITION BY month) AS no_home_runs
FROM statcast
WHERE events='home_run'
AND inning_topbot='Top'
AND game_date BETWEEN '2019-04-01' AND '2019-05-31';
': syntax error at or near "("
LINE 5:     SUM(events) RANK() OVER(PARTITION BY month) AS no_home_r...
                            ^


## Other ideas

In [256]:
# Practicing self join
sql_query = """
SELECT *
FROM player_id
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

   index name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0      0      abad   fernando     472551  abadf001   abadfe01           4994   
1      1     abreu      bryan     650556  abreb002  abreubr01          16609   
2      2     abreu       jose     547989  abrej003  abreujo02          15676   
3      3     acuna     ronald     660670  acunr001  acunaro01          18401   
4      4      adam      jason     592094  adamj002   adamja01          11861   

   mlb_played_first  mlb_played_last  
0            2010.0           2019.0  
1            2019.0           2019.0  
2            2014.0           2019.0  
3            2018.0           2019.0  
4            2018.0           2019.0  


In [1]:
# Practicing self join
sql_query = """
SELECT CONCAT(t1.name_last, ' ', t1.name_first)
FROM player_id AS t1, player_id AS t2
WHERE 
LIMIT 5;
"""

df_query = pd.read_sql_query(sql_query,con)
print(df_query)

NameError: name 'pd' is not defined